In [42]:
import torch
import torch.optim as optim
import torch.nn as nn
import torchvision.models as models
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import timm  # Import timm for NASNet

In [43]:
#Define transformations
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Custom dataset class to load images from directories
class GrapeLeafDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label

# Load dataset from Kaggle directory
data_dir = "Grapevine_Leaves_Image_Dataset"
classes = sorted(os.listdir(data_dir))

# Extract image paths and labels
image_paths, labels = [], []
for class_idx, class_name in enumerate(classes):
    class_dir = os.path.join(data_dir, class_name)
    for img_file in os.listdir(class_dir):
        image_paths.append(os.path.join(class_dir, img_file))
        labels.append(class_idx)

# Split into train and test sets
train_paths, test_paths, train_labels, test_labels = train_test_split(
    image_paths, labels, test_size=0.2, stratify=labels, random_state=42
)

# Create dataset objects
train_dataset = GrapeLeafDataset(train_paths, train_labels, transform=transform)
test_dataset = GrapeLeafDataset(test_paths, test_labels, transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [44]:
# Custom CNN Model
class CustomCNN(nn.Module):
    def __init__(self, num_classes):
        super(CustomCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 16 * 16, 512)
        self.fc2 = nn.Linear(512, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
def load_model(architecture, num_classes):
    architecture = architecture.lower()
    
    if architecture == "customcnn":  # ✅ Add CustomCNN model
        model = CustomCNN(num_classes)
    
    elif architecture == "resnet":
        model = models.resnet50(pretrained=True)
        model.fc = nn.Linear(model.fc.in_features, num_classes)

    elif architecture == "efficientnet":
        model = models.efficientnet_b0(pretrained=True)
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)

    elif architecture == "vgg":
        model = models.vgg16(pretrained=True)
        model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)

    elif architecture == "mobilenet":
        model = models.mobilenet_v3_large(pretrained=True)
        model.classifier[3] = nn.Linear(model.classifier[3].in_features, num_classes)

    elif architecture == "densenet":
        model = models.densenet121(pretrained=True)
        model.classifier = nn.Linear(model.classifier.in_features, num_classes)

    elif architecture == "nasnet":
        model = timm.create_model("nasnetalarge", pretrained=True)
        if hasattr(model, "classifier"):
            model.classifier = nn.Linear(model.classifier.in_features, num_classes)
        elif hasattr(model, "last_linear"):
            model.last_linear = nn.Linear(model.last_linear.in_features, num_classes)
        else:
            raise AttributeError("Could not find the classification layer in NASNetALarge.")

    else:
        raise ValueError(f"Unknown architecture: {architecture}")
    
    return model

In [46]:
# Training Settings
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = len(classes)
num_epochs = 5
criterion = nn.CrossEntropyLoss()

In [56]:
# Define models
models_list = {
    "CustomCNN": CustomCNN(num_classes).to(device),
    "ResNet": load_model("resnet", num_classes).to(device),
    "EfficientNet": load_model("efficientnet", num_classes).to(device),
    "VGG": load_model("vgg", num_classes).to(device),
    "MobileNet": load_model("mobilenet", num_classes).to(device),
    "DenseNet": load_model("densenet", num_classes).to(device),
    "NASNet": load_model("nasnet", num_classes).to(device)
}

c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IM

In [57]:
# Function to evaluate model
def evaluate_model(model, data_loader):
    model.eval()
    correct, total, loss_total = 0, 0, 0
    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss_total += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return loss_total / len(data_loader), 100 * correct / total

In [59]:
# Training Loop
for model_name, model in models_list.items():
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    print(f"\nTraining {model_name}...\n")

    for epoch in range(num_epochs):
        model.train()
        running_loss, correct, total = 0.0, 0, 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_loss = running_loss / len(train_loader)
        train_acc = 100 * correct / total
        val_loss, val_acc = evaluate_model(model, test_loader)

        print(f"Epoch [{epoch+1}/{num_epochs}] Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}% | Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")
    
    torch.save(model.state_dict(), f"{model_name}.pth")

print("Training complete!")


Training CustomCNN...

Epoch [1/5] Train Loss: 1.6164, Train Acc: 23.00% | Val Loss: 1.6072, Val Acc: 32.00%
Epoch [2/5] Train Loss: 1.5728, Train Acc: 28.25% | Val Loss: 1.5696, Val Acc: 25.00%
Epoch [3/5] Train Loss: 1.3981, Train Acc: 41.25% | Val Loss: 1.4184, Val Acc: 38.00%
Epoch [4/5] Train Loss: 1.1025, Train Acc: 57.75% | Val Loss: 1.4384, Val Acc: 39.00%
Epoch [5/5] Train Loss: 0.7839, Train Acc: 69.50% | Val Loss: 1.4060, Val Acc: 46.00%

Training ResNet...

Epoch [1/5] Train Loss: 1.5845, Train Acc: 41.75% | Val Loss: 13.5240, Val Acc: 22.00%
Epoch [2/5] Train Loss: 1.0425, Train Acc: 62.00% | Val Loss: 14.4105, Val Acc: 31.00%
Epoch [3/5] Train Loss: 0.6395, Train Acc: 75.75% | Val Loss: 2.2566, Val Acc: 55.00%
Epoch [4/5] Train Loss: 0.5994, Train Acc: 77.25% | Val Loss: 4.0221, Val Acc: 27.00%
Epoch [5/5] Train Loss: 0.4117, Train Acc: 85.75% | Val Loss: 1.4332, Val Acc: 62.00%

Training EfficientNet...

Epoch [1/5] Train Loss: 1.0465, Train Acc: 57.00% | Val Loss: 1.85

In [60]:
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
# Load trained models and evaluate accuracy
model_names = ["CustomCNN", "ResNet", "EfficientNet", "VGG", "MobileNet", "DenseNet", "NASNet"]
model_accuracies = {}

for model_name in model_names:
    model = load_model(model_name.lower(), num_classes)

    model_path = f"{model_name.lower()}.pth"
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"Model checkpoint '{model_path}' not found!")
    
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    model_accuracies[model_name] = accuracy
    print(f"{model_name} Test Accuracy: {accuracy:.4f}")

# Plot accuracy chart
plt.figure(figsize=(8, 6))
plt.bar(model_accuracies.keys(), model_accuracies.values(), color='skyblue')
plt.xlabel("Model")
plt.ylabel("Accuracy")
plt.title("Model Performance Comparison on Grapevine Dataset")
plt.xticks(rotation=45)
plt.tight_layout()

# Save plot
plt.savefig("grapevine_accuracy_plot.png")
plt.close()
print("Accuracy plot saved as grapevine_accuracy_plot.png")

C:\Users\HP\AppData\Local\Temp\ipykernel_22444\1723363588.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=devi

CustomCNN Test Accuracy: 0.4600


c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\HP\AppData\Local\Temp\ipykernel_22444\1723363588.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See

ResNet Test Accuracy: 0.6200


c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\HP\AppData\Local\Temp\ipykernel_22444\1723363588.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during u

...

MobileNet Test Accuracy: 0.3200


c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\HP\AppData\Local\Temp\ipykernel_22444\1723363588.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpicklin

DenseNet Test Accuracy: 0.7400


C:\Users\HP\AppData\Local\Temp\ipykernel_22444\1723363588.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=devi

NASNet Test Accuracy: 0.8200
Accuracy plot saved as grapevine_accuracy_plot.png


In [61]:
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
import numpy as np

class_names = classes

# Load Trained Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def load_trained_model(model_name):
    model = load_model(model_name, num_classes)
    model.load_state_dict(torch.load(f"{model_name}.pth", map_location=device))
    model.to(device)
    model.eval()  # Ensure the model is in evaluation mode
    return model

# ROC Curve Function
def plot_roc_curve(model, test_loader, num_classes, class_names):
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            probs = torch.nn.functional.softmax(outputs, dim=1)
            all_preds.extend(probs.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Binarize the labels for ROC curve
    all_labels_bin = label_binarize(all_labels, classes=np.arange(num_classes))
    all_preds = np.array(all_preds)

    # Plot ROC curve for each class
    plt.figure(figsize=(10, 8))
    for i in range(num_classes):
        fpr, tpr, _ = roc_curve(all_labels_bin[:, i], all_preds[:, i])
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f"Class {class_names[i]} (AUC = {roc_auc:.2f})")

    plt.plot([0, 1], [0, 1], linestyle='--', color='gray')  
    plt.title('Multi-Class ROC Curve')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend(loc='best')
    plt.tight_layout()

    # Save the plot
    roc_image_path = "roc_curve.png"
    plt.savefig(roc_image_path)
    plt.close()

    return roc_image_path

# Example Usage
print("Loading model...")
model_name = "resnet"  
model = load_trained_model(model_name)
roc_image_path = plot_roc_curve(model, test_loader, num_classes, class_names)
print("ROC Curve saved at", roc_image_path)

Loading model...


c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\HP\AppData\Local\Temp\ipykernel_22444\3385842657.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See

ROC Curve saved at roc_curve.png


In [62]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Load Trained Models
def load_trained_model(model_name, num_classes):
    model = load_model(model_name, num_classes)  # Pass num_classes dynamically
    model.load_state_dict(torch.load(f"{model_name}.pth", map_location=device))  # Fixed model loading
    model.to(device)
    model.eval()  # Ensure the model is in evaluation mode
    return model

# Confusion Matrix
def generate_confusion_matrix(model_name, model, test_loader, class_names):
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    cm = confusion_matrix(all_labels, all_preds)
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]  # Normalize confusion matrix

    # Plot confusion matrix
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm_normalized, annot=True, fmt=".2f", cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.title(f"Confusion Matrix for {model_name}")
    plt.ylabel("True Labels")
    plt.xlabel("Predicted Labels")
    plt.tight_layout()

    # Save confusion matrix plot as PNG
    plot_path = f"{model_name}_confusion_matrix.png"
    plt.savefig(plot_path)
    plt.close()

    return plot_path

# Choose a model and generate confusion matrix
print("Loading model...")
model_name = "resnet"  # or any model name you want to use
model = load_trained_model(model_name, num_classes)  # Pass num_classes dynamically

print("Generating confusion matrix...")
conf_matrix_image_path = generate_confusion_matrix(model_name, model, test_loader, class_names)

print(f"Confusion Matrix saved at {conf_matrix_image_path}")

Loading model...


c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\HP\AppData\Local\Temp\ipykernel_22444\2223757233.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See 

Generating confusion matrix...
Confusion Matrix saved at resnet_confusion_matrix.png
